In [3]:
import time
import sqlite3
import urllib.parse
import uuid
import re
from datetime import datetime
import numpy as np
from collections import Counter
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [4]:
def create_database():
    """Creates the SQLite database and table for storing product details."""
    #Connect to SQLite Data
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()

    # Drop existing table to fix data corruption
    cursor.execute('DROP TABLE IF EXISTS products')
    
    # Create table for storing product details
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            category_name TEXT,
            product_name TEXT,
            price TEXT,
            rating TEXT,
            reviews TEXT,
            product_url TEXT
        )
    ''')
    conn.commit()
    conn.close()

In [5]:
def save_to_database(products):
    """Save scraped products to database"""
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    
    for product in products:
        cursor.execute(
            "INSERT INTO products (category_name, product_name, price, rating, reviews, product_url) VALUES (?, ?, ?, ?, ?, ?)",
            (product['category'], product['name'], product['price'], product['rating'], product['reviews'], product['url'])
        )
    
    conn.commit()
    conn.close()
    print(f"Successfully saved {len(products)} products to database!")

In [6]:
def display_database_contents():
    """Display all contents from database with each product displayed separately"""
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM products')
    data_stored = cursor.fetchall()
    conn.close()
    
    print("\n" + "="*80)
    print("DATABASE CONTENTS")
    print("="*80)
    
    if data_stored:
        print("\nALL PRODUCT DETAILS:")
        print("-" * 80)
        
        for idx, row in enumerate(data_stored, 1):
            print(f"\n------------------- Product {idx} -------------------")
            print(f"ID: {row[0]}")
            print(f"🛒 Category: {row[1] if row[1] else 'N/A'}")
            print(f"📱 Name: {row[2] if row[2] else 'N/A'}")
            print(f"💰 Price: {row[3] if row[3] else 'N/A'}")
            print(f"⭐ Rating: {row[4] if row[4] else 'N/A'}")
            print(f"💬 Reviews: {row[5] if row[5] else 'N/A'}")
            print(f"🌐 URL: {row[6] if row[6] else 'N/A'}")
        
        print(f"\n{'-' * 80}")
        print(f"Total Products in Database: {len(data_stored)}")
    else:
        print("No data found in database.")
    
    print("="*80)

In [7]:
def create_comparison_database():
    """Create comparison table with ratings and reviews - UPDATED SCHEMA"""
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    
    # Drop existing table to add new columns
    cursor.execute('DROP TABLE IF EXISTS price_comparisons')
    
    cursor.execute('''
        CREATE TABLE price_comparisons (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            search_term TEXT,
            amazon_name TEXT,
            amazon_price TEXT,
            amazon_rating TEXT,
            amazon_reviews TEXT,
            amazon_url TEXT,
            flipkart_name TEXT,
            flipkart_price TEXT,
            flipkart_rating TEXT,
            flipkart_reviews TEXT,
            flipkart_url TEXT,
            price_difference TEXT,
            similarity_score REAL,
            created_at TEXT
        )
    ''')
    conn.commit()
    conn.close()

In [8]:
def save_comparison_to_database(search_term, amazon_product, flipkart_product, price_diff, similarity):
    """Save single comparison to database - COMPLETELY FIXED ORDER"""
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    
    try:
        # Ensure price_diff is a number before formatting
        if isinstance(price_diff, str):
            price_diff = 0.0  # Default if string
        
        cursor.execute("""
            INSERT INTO price_comparisons 
            (search_term, amazon_name, amazon_price, amazon_rating, amazon_reviews, amazon_url, 
             flipkart_name, flipkart_price, flipkart_rating, flipkart_reviews, flipkart_url, 
             price_difference, similarity_score, created_at) 
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            search_term,                        # Column 1: search_term
            amazon_product['name'],             # Column 2: amazon_name
            amazon_product['price'],            # Column 3: amazon_price 
            amazon_product['rating'],           # Column 4: amazon_rating        
            amazon_product['reviews'],           # Column 5: amazon_review
            amazon_product['url'],              # Column 6: amazon_url
            flipkart_product['name'],           # Column 7: flipkart_name
            flipkart_product['price'],          # Column 8: flipkart_price
            flipkart_product['rating'],         # Column 9: flipkart_rating
            flipkart_product['reviews'],         # Column 10: flipkart_review
            flipkart_product['url'],            # Column 11: flipkart_url
            f"₹{float(price_diff):.0f}",        # Column 12: price_difference
            float(similarity),                  # Column 13: similarity_score
            datetime.now().isoformat()          # Column 14: created_at
        ))
        conn.commit()
        print("✅ Comparison saved successfully")
    except Exception as e:
        print(f"❌ Database save error: {e}")
    finally:
        conn.close()

In [9]:
def display_price_comparisons():
    """Display all price comparisons from database"""
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM price_comparisons ORDER BY created_at DESC')
    data_stored = cursor.fetchall()
    conn.close()
    
    print("\n" + "="*100)
    print("🏪 PRICE COMPARISON DATABASE")
    print("="*100)
    
    if data_stored:
        for idx, row in enumerate(data_stored, 1):
            print(f"\n{'-'*80}")
            print(f"COMPARISON {idx} - Search: '{row[1]}'")
            print(f"📅 Date: {row[14]}")  # created_at
            print(f"🔗 Similarity Score: {row[13]:.2f}")  # similarity_score
            print(f"{'-'*80}")
            
            print(f"🛒 AMAZON:")
            print(f"  📱 Name: {row[2]}")  # amazon_name
            print(f"  💰 Price: {row[3]}")  # amazon_price
            print(f"  ⭐ Rating: {row[4]}")  # amazon_rating
            print(f"  💬 Reviews: {row[5]}")  # amazon_reviews
            print(f"  🌐 URL: {row[6][:60]}...")  # amazon_url
            
            print(f"\n🏬 FLIPKART:")
            print(f"  📱 Name: {row[7]}")  # flipkart_name
            print(f"  💰 Price: {row[8]}")  # flipkart_price
            print(f"  ⭐ Rating: {row[9]}")  # flipkart_rating
            print(f"  💬 Reviews: {row[10]}")  # flipkart_reviews
            print(f"  🌐 URL: {row[11][:60]}...")  # flipkart_url
            
            print(f"\n💵 PRICE DIFFERENCE: {row[12]}")  # price_difference
        
        print(f"\n{'='*100}")
        print(f"📊 Total Comparisons: {len(data_stored)}")
        print("="*100)
    else:
        print("⚠️ No price comparisons found in database.")


In [10]:
def view_database_options():
    """Display database viewing options"""
    print("\n" + "-"*50)
    print("📊 DATABASE VIEWER OPTIONS")
    print("-"*50)
    print("1. View Products Database")
    print("2. View Price Comparisons")
    print("3. Back to Main Menu")
    print("-"*50)
    
    choice = get_user_choice(1, 3)
    
    if choice == 1:
        display_database_contents()
    elif choice == 2:
        display_price_comparisons()
    # choice 3 returns to main menu automatically


In [11]:
def create_bulk_products_table():
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()

    # Drop existing table to fix data corruption
    cursor.execute('DROP TABLE IF EXISTS bulk_products')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS bulk_products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            search_term TEXT,
            product_name TEXT,
            brand_name TEXT,
            price TEXT,
            rating TEXT,
            reviews TEXT,
            product_url TEXT,
            extraction_id TEXT,
            page_number INTEGER,
            extraction_date TEXT
        )
    ''')
    conn.commit()
    conn.close()


In [12]:
def save_bulk_products_to_database(products):
    """Save bulk extracted products to the dedicated bulk_products table."""
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    for product in products:
        cursor.execute(
            """INSERT INTO bulk_products 
               (search_term, product_name, brand_name, price, rating, reviews, product_url, 
                extraction_id, page_number, extraction_date) 
               VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
            (
                product['search_term'],
                product['name'],
                product['brand_name'],
                product['price'],
                product['rating'],
                product['reviews'],
                product['url'],
                product['extraction_id'],
                product['page_number'],
                product['extraction_date']
            )
        )
    conn.commit()
    conn.close()
    print(f"✅ Successfully saved {len(products)} products to bulk_products database!")


In [13]:
def extract_product_data(category_name, driver, products_list):
    """Extract 10 products data from current page and add to products_list"""
    print(f"\n{'='*70}")
    print(f"EXTRACTING PRODUCTS FROM: {category_name}")
    print('='*70)

    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)
    
    # Scroll down a bit to load more products
    driver.execute_script("window.scrollBy(0, 500);")
    time.sleep(2)

    # Wait for product containers and get all of them
    product_containers = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((
                By.XPATH, "//div[contains(@class, 'GridItem-module__container')]"
            ))
        )

    print(f"Found {len(product_containers)} products initially")

    # If we don't have enough products, scroll and look for more
    if len(product_containers) < 10:
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(2)
        product_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'GridItem-module__container')]")
        print(f"Found {len(product_containers)} products after scrolling")

    # Store main window handle
    main_window = driver.current_window_handle
    
    # Process first 10 products
    for i, container in enumerate(product_containers[:3]):
        print(f"\n{'-'*50}")
        print(f"PROCESSING {category_name} - PRODUCT {i+1}/10")
        print('-'*50)
        
        # Scroll element into view
        driver.execute_script("arguments[0].scrollIntoView(true);", container)
        time.sleep(1)

        # Get basic product info from deals page
        try:
            name = container.find_element(By.XPATH, ".//p[contains(@class, 'ProductCard-module__title')]").text
        except:
            try:
                # Alternative way to get name
                name = container.find_element(By.XPATH, ".//a[contains(@class, 'ProductCard-module__cardContainingLink')]").text
            except:
                name = f"{category_name} Product {i+1}"

        # Get product URL
        try:
            product_link = container.find_element(By.XPATH, ".//a[contains(@class, 'ProductCard-module__cardContainingLink')]")
            url = product_link.get_attribute('href')
        except:
            print(f"Could not find URL for product {i+1}, skipping...")
            continue

        print(f"Product Name: {name[:50]}...")
        print(f"Opening product in new tab...")

        # Open product in new tab
        driver.execute_script(f"window.open('{url}', '_blank');")
        time.sleep(2)

        # Switch to new tab
        new_window = [handle for handle in driver.window_handles if handle != main_window][0]
        driver.switch_to.window(new_window)

        try:
            # Wait for product page to load and get details
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "productTitle"))
            )

            # Get price
            try:
                price_selectors = [
                    ".a-price-whole",
                    ".a-price .a-offscreen", 
                    "span[aria-label*='₹']",
                    ".a-text-price"
                ]
                
                for selector in price_selectors:
                    try:
                        price_elem = driver.find_element(By.CSS_SELECTOR, selector)
                        price_text = price_elem.text.strip() or price_elem.get_attribute('textContent').strip()
                        if price_text and any(char.isdigit() for char in price_text):
                            price = price_text
                            break
                    except:
                        continue     
            except:
                price = "Price not available"

            # Get rating
            try:
                rating_elem = driver.find_element(By.CSS_SELECTOR, "span.a-icon-alt")
                rating = rating_elem.text.strip() or rating_elem.get_attribute('textContent').strip()
            except:
                rating = "Rating not available"

            # Get review count
            try:
                reviews = driver.find_element(By.XPATH, "//span[@id='acrCustomerReviewText']").text
            except:
                reviews = "Reviews not available"

            # Store product details
            product_info = {
                'category': category_name,
                'name': name.strip(),
                'price': price,
                'url': url,
                'rating': rating,
                'reviews': reviews
            }

            products_list.append(product_info)
            
            # Display scraped details immediately
            print(f"✓ Successfully scraped product details:")
            print(f"  Category: {product_info['category']}")
            print(f"  Name: {product_info['name']}")
            print(f"  Price: {product_info['price']}")
            print(f"  Rating: {product_info['rating']}")
            print(f"  Reviews: {product_info['reviews']}")
            print(f"  URL: {product_info['url'][:50]}...")

        except Exception as e:
            print(f"Error scraping product {i+1}: {e}")

        # Close current tab and switch back to main window
        driver.close()
        driver.switch_to.window(main_window)
        time.sleep(2)

In [14]:
def todays_deals_scraper():
    """Scrape Today's Deals from Amazon - Modified existing code"""
    print("\n🎯Starting Today's Deals Scraping in Amazon...")
    # Setup ChromeDriver
    options = Options()
    options.add_argument('--start-maximized')
    driver = webdriver.Chrome(options=options)
    
    try:
        # Open Amazon and navigate to "Today's Deals"
        print("Opening Amazon India...")
        driver.get('https://www.amazon.in')
        time.sleep(2)
        
        print("Navigating to Today's Deals...")
        driver.find_element(By.LINK_TEXT, "Today's Deals").click()
        time.sleep(2)
    
        # Main products list to store all products from all categories
        all_products = []
        
        # Find category carousel
        try:
            print('Finding category buttons... ')
            categories = driver.find_elements(By.CSS_SELECTOR, "button[data-csa-c-type='uxElement'][data-csa-c-element-type='option']")
            print(f"Found {len(categories)} categories")
            print("\nAvailable Categories:")
            for c in categories:
                print(c.text)
            
            # Process first 6 categories
            for i, category in enumerate(categories[:6]):
                try:
                    # Get category name
                    category_name = category.text
                    if not category_name:
                        category_name = f"Category {i+1}"
                    
                    print(f"\n{'*'*80}")
                    print(f"PROCESSING CATEGORY {i+1}/6: {category_name}")
                    print('*'*80)
                    
                    # Click on category
                    category.click()
                    time.sleep(3)
                    
                    # Extract products from this category
                    extract_product_data(category_name, driver, all_products)
      
                    print(f"✅ Completed category: {category_name}")
                    
                except Exception as e:
                    print(f"❌ Error with category {i+1}: {e}")
        
        except Exception as e:
            print(f"❌ Error finding categories: {e}")
    
        # Final summary of all scraped products
        print(f"\n" + "="*80)
        print(f"🎉 SCRAPING COMPLETE! Successfully scraped {len(all_products)} products total")
        print("="*80)
    
        if all_products:
            # Save to database
            print(f"\n💾 Saving {len(all_products)} products to database...")
            save_to_database(all_products)
            
            # Ask user if they want to view the scraped data
            view_choice = input("\nWould you like to view the scraped products? (y/n): ").lower()
            if view_choice == 'y':
                display_database_contents()
        else:
            print("⚠️ No products were successfully scraped.")
    
    except TimeoutException:
        print("⏰ Page load Timeout. Please check your internet connection.")
    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")
    finally:
        driver.quit()
        print("\n🔄 Returning to main menu...")
        

In [35]:
def extract_amazon_search_data(search_term, driver, products_list):
    """Extract 10 products data from Amazon search results without opening individual product pages"""
    print(f"\n{'='*70}")
    print(f"EXTRACTING PRODUCTS FROM AMAZON SEARCH: {search_term}")
    print('='*70)

    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)
    
    # Scroll down to load more products
    driver.execute_script("window.scrollBy(0, 500);")
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-id]"))
    )

    # Wait for search result containers
    try:
        product_containers = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@data-component-type='s-search-result']"))
        )
    except:
        print("❌ No search results found!")
        return

    print(f"Found {len(product_containers)} products initially")

    # If we don't have enough products, scroll and look for more
    if len(product_containers) < 10:
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(2)
        product_containers = driver.find_elements(By.XPATH, "//div[@data-component-type='s-search-result']")
        print(f"Found {len(product_containers)} products after scrolling")
    
    # Process first 10 products directly from search results
    for i, container in enumerate(product_containers[:10]):
        print(f"\n{'-'*50}")
        print(f"PROCESSING SEARCH RESULT {i+1}/10")
        print('-'*50)
        
        # Scroll element into view
        driver.execute_script("arguments[0].scrollIntoView(true);", container)
        time.sleep(1)

        try:
            # Get product name
            title_container = container.find_element(By.XPATH, ".//div[@data-cy='title-recipe']")
            try:
                name_elem = title_container.find_element(By.XPATH, ".//a//h2/span")
                name = name_elem.text.strip()
            except:
                try:
                    name_elem = title_container.find_element(By.XPATH, ".//h2//span")
                    name = name_elem.text.strip()
                except:
                    name = f"Search Product {i+1}"

            # Get product URL
            try:
                product_link = container.find_element(
                    By.XPATH, 
                    ".//div[@data-cy='title-recipe']//a[contains(@class, 's-line-clamp')][@href]"
                )
                url = product_link.get_attribute('href')
                if 'javascript:void(0)' not in url and ('/dp/' in url or '/sspa/click' in url):
                    if not url.startswith('http'):
                        url = 'https://www.amazon.in' + url
                else:
                    raise Exception("Found sponsored label link instead of product link")
                    
            except:
                print(f"❌ Could not find valid product URL for item {i+1}, skipping...")
                continue

            # Get price from search results
            try:
                price_elem = container.find_element(By.XPATH, ".//span[@class='a-price-whole']")
                price = price_elem.text.strip()
            except:
                price = "Price not available"

            # Get rating from search results
            try:
                rating_elem = container.find_element(By.XPATH, ".//span[contains(@class, 'a-icon-alt')]")
                rating_text = rating_elem.text.strip() or rating_elem.get_attribute('aria-label') or rating_elem.get_attribute('textContent').strip()
                if rating_text and ("out of" in rating_text or "stars" in rating_text.lower()):
                    rating = rating_text
                else:
                    rating = "Rating not available"
            except:
                rating = "Rating not available"

            # Get reviews count from search results
            try:
                review_elem = container.find_element(By.XPATH, ".//a[contains(@href, '#customerReviews')]//span")
                review_text = review_elem.text.strip() or review_elem.get_attribute('aria-label')
                if review_text and ('rating' in review_text.lower() or 'review' in review_text.lower() or any(char.isdigit() for char in review_text)):
                    reviews = review_text
            except:
                reviews = "Reviews not available"

            # Store product details
            product_info = {
                'category': f"Search: {search_term}",
                'name': name.strip(),
                'price': price,
                'url': url,
                'rating': rating,
                'reviews': reviews
            }

            products_list.append(product_info)
            
            # Display scraped details immediately
            print(f"✅ Successfully scraped product details:")
            print(f"  Category: {product_info['category']}")
            print(f"  Name: {product_info['name'][:80]}...")
            print(f"  Price: {product_info['price']}")
            print(f"  Rating: {product_info['rating']}")
            print(f"  Reviews: {product_info['reviews']}")
            print(f"  URL: {product_info['url'][:60]}...")

        except Exception as e:
            print(f"❌ Error scraping product {i+1}: {e}")
            continue

    print(f"\n{'='*70}")
    print(f"✅ Finished extracting from Amazon search: {search_term}")
    print(f"📊 Successfully extracted {len([p for p in products_list if p['category'] == f'Search: {search_term}'])} products")
    print('='*70)


In [16]:
def search_amazon_only():
    """Search for user-specified products on Amazon and extract first 10 results"""
    print("\n🔍 Starting Amazon Product Search...")
    
    # Get search term from user
    search_term = input("Enter the product you want to search for: ").strip()
    if not search_term:
        print("❌ Search term cannot be empty!")
        return

    print(f"🔎 Searching for: '{search_term}' on Amazon...")
    
    # Setup ChromeDriver and navigate to search
    options = Options()
    options.add_argument('--start-maximized')
    driver = webdriver.Chrome(options=options)
    
    try:
        # Open Amazon and search
        driver.get('https://www.amazon.in')
        
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "twotabsearchtextbox"))
        )
        search_box.clear()
        search_box.send_keys(search_term)
        driver.find_element(By.ID, "nav-search-submit-button").click()

        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@data-component-type='s-search-result']"))
        )
        
        # Call existing function with search page type
        all_products = []
        extract_amazon_search_data(search_term, driver, all_products)
        
        # Save and display results
        if all_products:
            print(f"\n💾 Saving {len(all_products)} products to database...")
            save_to_database(all_products)
    
    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")
    finally:
        driver.quit()
        if input("\nWould you like to view the scraped products? (y/n): ").lower() == 'y':
                display_database_contents()
        print("\n🔄 Returning to main menu...")


In [40]:
# from difflib import SequenceMatcher
import difflib
from datetime import datetime

def calculate_similarity_score(text1, text2):
    """Calculate similarity between two strings using difflib"""
    return difflib.SequenceMatcher(None, text1.lower(), text2.lower()).ratio()

def extract_price_number(price_text):
    """Extract numeric value from price text"""
    if not price_text or not isinstance(price_text, str):
        return 0.0
    
    # Remove everything except digits and dots
    numbers = re.findall(r'\d+', price_text.replace(',', ''))
    if numbers:
        return float(''.join(numbers))
    return 0.0

def clean_amazon_name_for_flipkart_search(amazon_product_name):
    """Clean Amazon product name for Flipkart search by removing unnecessary symbols"""
    # Remove commas, pipes, and other special characters
    cleaned_name = re.sub(r'[|,;:()[\]{}-]+', ' ', amazon_product_name)
    # Replace multiple spaces with single space
    cleaned_name = re.sub(r'\s+', ' ', cleaned_name)
    # Remove extra whitespace
    cleaned_name = cleaned_name.strip()
    return cleaned_name


def display_comparison_results():
    """Display price comparison results"""
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM price_comparisons ORDER BY id DESC LIMIT 10')
    comparisons = cursor.fetchall()
    conn.close()
    
    if not comparisons:
        print("📊 No comparison data found!")
        return
    
    print("\n" + "="*100)
    print("PRICE COMPARISON RESULTS")
    print("="*100)
    
    for idx, row in enumerate(comparisons, 1):
        print(f"\n{'-'*80}")
        print(f"COMPARISON {idx}")
        print(f"{'-'*80}")
        try:
            print(f"Search Term: {row[1]}")    
            print(f"🛒 AMAZON:")
            print(f"  📱 Product: {row[2]}")
            print(f"  💰 Price: {row[3]}")  
            print(f"  ⭐ Rating: {row[4]}") 
            print(f"  💬 Reviews: {row[5]}") 
            print(f"\n🏬 FLIPKART:")
            print(f"  📱 Product: {row[7]}")  
            print(f"  💰 Price: {row[8]}")
            print(f"  ⭐ Rating: {row[9]}")
            print(f"  💬 Reviews: {row[10]}")
            print(f"\n💵 Price Difference: {row[12]}") 
            try:
                similarity = float(row[13])
                print(f"🔗 Similarity Score: {similarity:.3f}") 
            except:
                print(f"🔗 Similarity Score: {row[13]}")  
            print(f"Date: {row[14][:19].replace('T', ' ')}") 
        except Exception as e:
            print(f"❌ Error displaying comparison {idx}: {e}")

In [18]:
def search_flipkart_for_amazon_product(amazon_product_name, driver):
    """Search Flipkart using cleaned exact Amazon product name and return 10 products"""
    print(f"\n🔍 Searching Flipkart for: {amazon_product_name[:50]}...")
    
    try:
        # Clean the Amazon product name for Flipkart search
        cleaned_name = clean_amazon_name_for_flipkart_search(amazon_product_name)
        print(f"   🧹  Search term: {cleaned_name}...")
        
        # Navigate to Flipkart search with cleaned exact Amazon product name
        search_url = f"https://www.flipkart.com/search?q={urllib.parse.quote(cleaned_name)}"
        print(f"   🌐 Search URL: {search_url}...")
        driver.get(search_url)
        time.sleep(2)

        # Check if page loaded correctly
        page_title = driver.title.lower()
        if "error" in page_title or "repair" in page_title or "sorry" in page_title:
            print("❌ Flipkart page shows error - trying with shorter search term")
            # Try with first 8 words only as fallback
            short_search = " ".join(cleaned_name.split()[:8])
            search_url = f"https://www.flipkart.com/search?q={urllib.parse.quote(short_search)}"
            driver.get(search_url)
            time.sleep(2)
            
        # Wait for page to fully load
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        
        # Try different container selectors
        container_selectors = [
            "div[data-id]",
            "div._4ddWXP", 
            "div._2kHMtA"
        ]
        
        product_containers = []
        for selector in container_selectors:
            containers = driver.find_elements(By.CSS_SELECTOR, selector)
            if containers:
                print(f"   ✅ Found {len(containers)} containers using: {selector}")
                product_containers = containers
                break
        
        if not product_containers:
            print("❌ No product containers found with any selector")
            return []
        
        flipkart_products = []
        main_window = driver.current_window_handle
        
        for i, container in enumerate(product_containers[:20]):
            if len(flipkart_products) >= 10:
                break
                
            try:
                # Get name
                name = None
                try:
                    name_elem = container.find_element(By.CSS_SELECTOR, "a[title]")
                    name = name_elem.get_attribute('title')
                except:
                    try:
                        name_elem = container.find_element(By.CSS_SELECTOR, "div._4rR01T")
                        name = name_elem.text.strip()
                    except:
                        continue
                
                if not name or len(name) < 5:
                    continue
                
                # Get price
                price = "Price not available"
                try:
                    price_elem = container.find_element(By.CSS_SELECTOR, "div.Nx9bqj")
                    price = price_elem.text.strip()
                except:
                    try:
                        price_elem = container.find_element(By.CSS_SELECTOR, "div._30jeq3")
                        price = price_elem.text.strip()
                    except:
                        pass
                
                # Get URL
                url = None
                try:
                    link_elem = container.find_element(By.CSS_SELECTOR, "a[href*='/p/']")
                    url = link_elem.get_attribute('href')
                    if not url.startswith('http'):
                        url = 'https://www.flipkart.com' + url
                except:
                    continue

                # Initialize flags and default values
                rating = "Rating not available"
                reviews = "Reviews not available"
                rating_found = False

                #Get rating
                rating_selectors = [
                    "div.XQDdHH"
                ]
                
                for rating_sel in rating_selectors:
                    try:
                        rating_elem = container.find_element(By.CSS_SELECTOR, rating_sel)
                        rating_text = rating_elem.text.strip()
                        if rating_text and (rating_text.replace('.', '').isdigit() or 'star' in rating_text.lower()):
                            rating = rating_text
                            rating_found = True
                            break
                    except:
                        continue
                        
                try:
                    # Open product page in new tab
                    driver.execute_script(f"window.open('{url}', '_blank');")
                    time.sleep(0.5)
                    
                    # Switch to new tab
                    driver.switch_to.window(driver.window_handles[-1])
                    
                    # Wait for product page to load
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.TAG_NAME, "body"))
                    )
                    
                    # Try to get rating from product page if not found on search page
                    if not rating_found:
                        product_page_rating_selectors = [
                            "div.XQDdHH",        # Same as search results
                            "span._1lRcqv",      # Product page rating
                            "div._3LWZlK",       # Legacy rating
                            "div._13rOPH"        # Alternative rating
                        ]
                        
                        for rating_sel in product_page_rating_selectors:
                            try:
                                rating_elem = driver.find_element(By.CSS_SELECTOR, rating_sel)
                                rating_text = rating_elem.text.strip()
                                if rating_text and (rating_text.replace('.', '').isdigit() or 'star' in rating_text.lower()):
                                    rating = rating_text
                                    break
                            except:
                                continue
                    
                    # Always try to get reviews from product page
                    product_page_reviews_selectors = [
                        "span._2_R_DZ",             # Common product page reviews
                        "span.Wphh3N",              # Same as search results  
                        "span._13vcmD",             # Alternative reviews selector
                        "span[class*='_2_R_DZ']"    # Partial match
                    ]
                    
                    for review_sel in product_page_reviews_selectors:
                        try:
                            review_elem = driver.find_element(By.CSS_SELECTOR, review_sel)
                            review_text = review_elem.text.strip()
                            
                            # Validate and extract review text
                            if review_text and any(char.isdigit() for char in review_text) and 'review' in review_text.lower():
                                import re
                                if '&' in review_text and 'rating' in review_text.lower():
                                    # Extract only review part from "11 Ratings & 0 Reviews"
                                    match = re.search(r'([\d,]+)\s*Reviews?', review_text, re.IGNORECASE)
                                    if match:
                                        reviews = f"{match.group(1)} Reviews"
                                    else:
                                        continue
                                else:
                                    reviews = review_text
                                break
                        except:
                            continue

                    driver.close()
                    driver.switch_to.window(main_window)
                    time.sleep(1)
                    
                except Exception as e:
                    print(f"   ❌ Error accessing product page: {e}")
                    try:
                        driver.switch_to.window(main_window)
                    except:
                        pass
                
                if name and url:
                    flipkart_products.append({
                        'category': f"Flipkart Search: {cleaned_name}",
                        'name': name,
                        'price': price,
                        'url': url,
                        'rating': rating,
                        'reviews': reviews
                    })
                    print(f"   ✅ Added product {len(flipkart_products)}: {name[:50]}...")
            
            except Exception as e:
                print(f"   ❌ Error processing product {i}: {e}")
                try:
                    driver.switch_to.window(main_window)
                except:
                    pass
                continue
        
        print(f"✅ Successfully found {len(flipkart_products)} Flipkart products")
        return flipkart_products
        
    except Exception as e:
        print(f"❌ Error searching Flipkart: {e}")
        return []


In [19]:
def compare_amazon_flipkart():
    """Compare Amazon and Flipkart prices"""
    print("\n⚖️ Starting Amazon vs Flipkart Price Comparison...")
    
    search_term = input("Enter the product you want to compare: ").strip()
    if not search_term:
        print("❌ Search term cannot be empty!")
        return
    
    options = Options()
    options.add_argument('--start-maximized')
    driver = webdriver.Chrome(options=options)
    
    try:
        # Get Amazon products
        print(f"\n🔍 Searching Amazon for '{search_term}'...")
        driver.get('https://www.amazon.in')
        time.sleep(2)
        
        search_box = driver.find_element(By.ID, "twotabsearchtextbox")
        search_box.clear()
        search_box.send_keys(search_term)
        driver.find_element(By.ID, "nav-search-submit-button").click()
        time.sleep(3)
        
        amazon_products = []
        extract_amazon_search_data(search_term, driver, amazon_products)
        
        # Save and display results
        if amazon_products:
            print(f"\n💾 Saving {len(amazon_products)} products to database...")
            save_to_database(amazon_products)
        
        if not amazon_products:
            print("❌ No Amazon products found!")
            return
        
        successful_comparisons = 0
        
        for i, amazon_product in enumerate(amazon_products[:3]):
            print(f"\n{'-'*80}")
            print(f"COMPARING AMAZON PRODUCT {i+1}/3")
            print('-'*80)
            print(f"Amazon Product: {amazon_product['name'][:80]}...")
            print(f"Amazon Price: {amazon_product['price']}")
            
            flipkart_products = search_flipkart_for_amazon_product(amazon_product['name'], driver)

            # Save and display results
            if flipkart_products:
                print(f"\n💾 Saving {len(flipkart_products)} products to database...")
                save_to_database(flipkart_products)
                
            if not flipkart_products:
                print(f"⚠️ Skipping - No Flipkart products found")
                continue
            
            # Find best match
            best_match = None
            highest_score = 0.0
            best_index = -1
            
            print(f"\n📊 Calculating similarity scores:")
            for j, flipkart_product in enumerate(flipkart_products):
                score = calculate_similarity_score(amazon_product['name'], flipkart_product['name'])
                print(f"   Product {j+1}: Score {score:.3f} - {flipkart_product['name'][:50]}...")
                
                if score > highest_score:
                    highest_score = score
                    best_match = flipkart_product
                    best_index = j
            
            if not best_match or highest_score < 0.3:
                print(f"❌ No good match found (highest score: {highest_score:.3f})")
                continue
            
            # Calculate price difference
            amazon_price_num = extract_price_number(amazon_product['price'])
            flipkart_price_num = extract_price_number(best_match['price'])
            price_difference = abs(amazon_price_num - flipkart_price_num)
            
            # Display comparison
            print(f"\n✅ BEST MATCH FOUND: Product {best_index+1} with Similarity Score {highest_score:.3f})")
            print(f"🛒 Amazon: {amazon_product['name'][:60]}...")
            print(f"   💰 Price: {amazon_product['price']}")
            print(f"🛍️ Flipkart: {best_match['name'][:60]}...")
            print(f"   💰 Price: {best_match['price']}")
            print(f"📊 Absolute Price Difference: ₹{price_difference:.0f}")
            
            # Store in database
            save_comparison_to_database(search_term, amazon_product, best_match, price_difference, highest_score)
            successful_comparisons += 1
        
        print(f"\n" + "="*80)
        print(f"🎉 COMPARISON COMPLETE!")
        print(f"📝 Search Term: '{search_term}'")
        print(f"✅ Successfully compared {successful_comparisons} products")
        print("="*80)
        
        if successful_comparisons > 0:
            view_choice = input("\nWould you like to view the comparison results? (y/n): ").lower()
            if view_choice == 'y':
                display_comparison_results()
    
    except Exception as e:
        print(f"❌ Error during comparison: {e}")
    finally:
        driver.quit()
        print("\n🔄 Returning to main menu...")


In [20]:
def show_top_rated_bulk_products(extraction_id):
    """Top 10 Highest Rated"""
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    cursor.execute("""
        SELECT product_name, price, rating, reviews, product_url
        FROM bulk_products 
        WHERE extraction_id = ? AND rating NOT NULL AND rating != ''
        ORDER BY 
            CAST(replace(substr(rating, 1, instr(rating, ' ')-1), ',', '') AS FLOAT) DESC
        LIMIT 10
    """, (extraction_id,))
    rows = cursor.fetchall()
    conn.close()
    print(f"\n⭐ TOP 10 HIGHEST RATED PRODUCTS")
    for i, row in enumerate(rows, 1):
        print(f"\n------------------- Product {i} -------------------")
        print(f"📱 Name: {row[0]}")
        print(f"💰 Price: ₹{row[1]}")
        print(f"⭐ Rating: {row[2]}")
        print(f"💬 Reviews: {row[3]}")
        print(f"🌐 URL: {row[4]}")

def show_lowest_price_bulk_products(extraction_id):
    """Top 10 Lowest Priced"""
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    cursor.execute("""
        SELECT product_name, price, rating, reviews, product_url
        FROM bulk_products
        WHERE extraction_id = ? AND price NOT NULL AND price != '' AND price != 'Price not available'
        ORDER BY 
            CAST(replace(price, ',', '') AS FLOAT) ASC
        LIMIT 10
    """, (extraction_id,))
    rows = cursor.fetchall()
    conn.close()
    print(f"\n💰 TOP 10 LOWEST PRICED PRODUCTS")
    print("="*60)
    for i, row in enumerate(rows, 1):
        print(f"\n------------------- Product {i} -------------------")
        print(f"📱 Name: {row[0]}")
        print(f"💰 Price: ₹{row[1]}")
        print(f"⭐ Rating: {row[2]}")
        print(f"💬 Reviews: {row[3]}")
        print(f"🌐 URL: {row[4]}")


def show_most_reviewed_bulk_products(extraction_id):
    """Top 10 Most Reviewed"""
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    cursor.execute("""
        SELECT product_name, price, rating, reviews, product_url
        FROM bulk_products
        WHERE extraction_id = ? AND reviews NOT NULL AND reviews != ''
        ORDER BY 
            CAST(replace(replace(replace(reviews, ',', ''), 'Reviews', ''), 'ratings', '') AS INTEGER) DESC
        LIMIT 10
    """, (extraction_id,))
    rows = cursor.fetchall()
    conn.close()
    print(f"\n💬 TOP 10 MOST REVIEWED PRODUCTS")
    print("="*60)
    for i, row in enumerate(rows, 1):
        print(f"\n------------------- Product {i} -------------------")
        print(f"📱 Name: {row[0]}")
        print(f"💰 Price: ₹{row[1]}")
        print(f"⭐ Rating: {row[2]}")
        print(f"💬 Reviews: {row[3]}")
        print(f"🌐 URL: {row[4]}")

def show_bulk_products_in_price_range(extraction_id):
    """Custom Price Range"""
    try:
        low = float(input("Enter minimum price: "))
        high = float(input("Enter maximum price: "))
    except:
        print("Invalid input!")
        return
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    cursor.execute("""
        SELECT product_name, price, rating, reviews, product_url
        FROM bulk_products
        WHERE extraction_id = ?
        AND price NOT NULL AND price != '' AND price != 'Price not available'
        AND CAST(replace(price, ',', '') AS FLOAT) BETWEEN ? AND ?
        ORDER BY CAST(replace(price, ',', '') AS FLOAT) ASC
        LIMIT 20
    """, (extraction_id, low, high))
    rows = cursor.fetchall()
    conn.close()
    print(f"\n💸 PRODUCTS BETWEEN ₹{low:,.0f} AND ₹{high:,.0f}")
    print("="*60)
    for i, row in enumerate(rows, 1):
        print(f"\n------------------- Product {i} -------------------")
        print(f"📱 Name: {row[0]}")
        print(f"💰 Price: ₹{row[1]}")
        print(f"⭐ Rating: {row[2]}")
        print(f"💬 Reviews: {row[3]}")
        print(f"🌐 URL: {row[4]}")

def show_bulk_price_distribution(extraction_id):
    """Show price distribution in percentile ranges"""
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    cursor.execute("""
        SELECT price FROM bulk_products 
        WHERE extraction_id = ? AND price != '' AND price != 'Price not available'
    """, (extraction_id,))

    prices = []
    for row in cursor.fetchall():
        try:
            price_str = str(row[0]).replace('₹', '').replace(',', '').strip()
            if price_str and price_str.replace('.', '', 1).isdigit():
                price_val = float(price_str)
                if price_val > 0:
                    prices.append(price_val)
        except:
            continue
    conn.close()
    if not prices:
        print("❌ No valid price data found for this extraction.")
        return
    
    # Create percentile bins
    bins = np.percentile(prices, [0, 20, 40, 60, 80, 100])
    counts = Counter(np.digitize(prices, bins, right=True))
    
    print(f"\n💰 PRICE DISTRIBUTION ({len(prices)} products)")
    print("="*60)
    for i in range(1, len(bins)):
        count = counts.get(i, 0)
        percentage = (count / len(prices)) * 100
        print(f"💸 ₹{int(bins[i-1]):,} - ₹{int(bins[i]):,}: {count} products ({percentage:5.1f}%)")

def show_bulk_rating_distribution(extraction_id):
    """Rating distribution"""
    from collections import Counter
    import sqlite3
    import re
    
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    cursor.execute("SELECT rating FROM bulk_products WHERE extraction_id = ?", (extraction_id,))
    all_ratings = [row[0] for row in cursor.fetchall()]
    conn.close()
    
    if not all_ratings:
        print("❌ No ratings found.")
        return
    
    rating_counts = Counter(all_ratings)
    
    print(f"\n⭐ RATING DISTRIBUTION ({len(all_ratings)} products)")
    print("="*60)
    
    for rating, count in rating_counts.most_common():
        percentage = (count / len(all_ratings)) * 100
        print(f"⭐ {rating:<30}: {count:3d} products ({percentage:5.1f}%)")
        
def show_bulk_brand_frequency(extraction_id):
    """Show actual brand frequency from dedicated brand column"""
    from collections import Counter
    
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    cursor.execute("""
        SELECT brand_name FROM bulk_products 
        WHERE extraction_id = ? AND brand_name != '' AND brand_name != 'Unknown Brand'
    """, (extraction_id,))
    
    brands = [row[0] for row in cursor.fetchall() if row]
    conn.close()
    
    if not brands:
        print("❌ No brand data found for this extraction.")
        return
    
    counter = Counter(brands)
    print(f"\n🏷️ BRAND FREQUENCY ANALYSIS ({len(brands)} products with brands)")
    print("="*60)
    for brand, count in counter.most_common(20):
        percentage = (count / len(brands)) * 100
        print(f"🏷️ {brand:<25}: {count:3d} products ({percentage:5.1f}%)")

def show_bulk_avg_price_by_rating(extraction_id):
    """Show average price for each rating level"""
    
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    cursor.execute("SELECT rating, price FROM bulk_products WHERE extraction_id = ?", (extraction_id,))
    all_data = cursor.fetchall()
    conn.close()

    # Group by rating (even if not numeric)
    rating_prices = {}
    for rating, price in all_data:
        try:
            price_str = str(price).replace('₹', '').replace(',', '').strip()
            price_cleaned = re.sub(r'[^\d.]', '', price_str)
            if price_cleaned and price_cleaned.replace('.', '', 1).isdigit():
                price_val = float(price_cleaned)
                if price_val > 0:
                    rating_key = str(rating)
                    if rating_key not in rating_prices:
                        rating_prices[rating_key] = []
                    rating_prices[rating_key].append(price_val)
        except:
            continue
    
    if not rating_prices:
        print("❌ No valid price data found.")
        return
    
    # Calculate averages for each rating
    print(f"\n💰⭐ AVERAGE PRICE BY RATING ({len(all_data)} total products)")
    print("="*60)
    
    for rating, prices in rating_prices.items():
        avg_price = sum(prices) / len(prices)
        min_price = min(prices)
        max_price = max(prices)
        
        print(f"⭐ {rating:<25}: ₹{avg_price:,.0f}")
        print(f"{'':25}  (₹{min_price:,.0f} - ₹{max_price:,.0f}, {len(prices)} products)")
        print()
    
    # Overall price statistics
    all_prices = [price for prices in rating_prices.values() for price in prices]
    if all_prices:
        print(f"📊 OVERALL PRICE STATISTICS:")
        print(f"💰 Average Price: ₹{sum(all_prices)/len(all_prices):,.0f}")
        print(f"💸 Cheapest: ₹{min(all_prices):,.0f}")
        print(f"💎 Most Expensive: ₹{max(all_prices):,.0f}")



def show_bulk_products_with_review_threshold(extraction_id):
    """Products with Reviews > X"""
    try:
        threshold = int(input("Show products with reviews greater than: "))
    except:
        print("Invalid input!")
        return
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    cursor.execute("""
        SELECT product_name, price, rating, reviews, product_url
        FROM bulk_products
        WHERE extraction_id = ? AND
        CAST(replace(replace(replace(reviews, ',', ''), 'Reviews', ''), 'ratings', '') AS INTEGER) > ?
        ORDER BY CAST(replace(replace(replace(reviews, ',', ''), 'Reviews', ''), 'ratings', '') AS INTEGER) DESC
        LIMIT 10
    """, (extraction_id, threshold))
    rows = cursor.fetchall()
    conn.close()
    print(f"\n💬 PRODUCTS WITH MORE THAN {threshold} REVIEWS")
    print("="*60)
    for i, row in enumerate(rows, 1):
        print(f"\n------------------- Product {i} -------------------")
        print(f"📱 Name: {row[0]}")
        print(f"💰 Price: ₹{row[1]}")
        print(f"⭐ Rating: {row[2]}")
        print(f"💬 Reviews: {row[3]}")
        print(f"🌐 URL: {row[4]}")

def show_bulk_products_by_brand(extraction_id):
    """Filter and show products by specific brand name using dedicated brand column"""
    brand_name = input("Enter brand name to filter by: ").strip()
    if not brand_name:
        print("❌ Brand name cannot be empty!")
        return
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    # Exact match and partial match search
    cursor.execute("""
        SELECT product_name, brand_name, price, rating, reviews, product_url 
        FROM bulk_products 
        WHERE extraction_id = ? AND (brand_name LIKE ? OR brand_name LIKE ?)
        ORDER BY brand_name, product_name LIMIT 30
    """, (extraction_id, brand_name, f'%{brand_name}%'))
    
    rows = cursor.fetchall()
    conn.close()
    if not rows:
        print(f"❌ No products found for brand '{brand_name}'.")
        return
    
    print(f"\n🏷️ PRODUCTS FROM BRAND: {brand_name.upper()}")
    print("="*80)
    for i, row in enumerate(rows, 1):
        print(f"\n------------------- Product {i} -------------------")
        print(f"📱 Brand: {row[1]}")
        print(f"📱 Name: {row[0]}")
        print(f"💰 Price: ₹{row[2]}")
        print(f"⭐ Rating: {row[3]}")
        print(f"💬 Reviews: {row[4]}")
        print(f"🌐 URL: {row[5]}")
    

def export_bulk_result_to_csv(extraction_id, search_term):
    """Export to CSV"""
    import csv
    conn = sqlite3.connect('amazon.db')
    cursor = conn.cursor()
    cursor.execute("""
        SELECT product_name, brand_name, price, rating, reviews, product_url, page_number
        FROM bulk_products 
        WHERE extraction_id = ?
    """, (extraction_id,))
    rows = cursor.fetchall()
    filename = f"{search_term.replace(' ', '_')}_bulk_{extraction_id}.csv"
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Product Name','Brand Name', 'Price', 'Rating', 'Reviews', 'URL', 'Page Number'])
        writer.writerows(rows)
    print(f"\n📄 CSV EXPORT COMPLETED")
    print("="*40)
    print(f"📁 Filename: {filename}")
    print(f"📊 Total Products: {len(rows)}")
    print(f"💾 File Created Successfully!")
    conn.close()


In [21]:
def bulk_analysis_menu(extraction_id, search_term):
    """Interactive analysis menu for a given bulk extraction."""
    import sqlite3

    while True:
        print(f"\n📊 BULK PRODUCT ANALYSIS MENU")
        print(f"🔍 Search Term: {search_term}")
        print(f"🆔 Extraction ID: {extraction_id}")
        print("="*55)
        # Quick stats
        conn = sqlite3.connect('amazon.db')
        cursor = conn.cursor()
        cursor.execute(
            "SELECT COUNT(*) FROM bulk_products WHERE extraction_id = ?", (extraction_id,))
        total_products = cursor.fetchone()[0]
        conn.close()
        print(f"📦 Total Products: {total_products}")

        # print(f"\n📊 BULK PRODUCT ANALYSIS OPTIONS")
        print("="*55)
        print("1. Top 10 Highest Rated Products")
        print("2. Top 10 Lowest Priced Products")
        print("3. Top 10 Most Reviewed Products")
        print("4. Products in Price Range (Custom)")
        print("5. Show Price Distribution (Count per Range)")
        print("6. Show Rating Distribution (Count per Level)")
        print("7. Brand Frequency Table")
        print("8. Average Price by Rating")
        print("9. Products with Reviews > X")
        print("10. Filter by Brand Name")
        print("11. Export Result to CSV")
        print("12. Back to Main Menu")
        print("="*55)

        choice = get_user_choice(1, 12)

        if choice == 1:
            show_top_rated_bulk_products(extraction_id)
        elif choice == 2:
            show_lowest_price_bulk_products(extraction_id)
        elif choice == 3:
            show_most_reviewed_bulk_products(extraction_id)
        elif choice == 4:
            show_bulk_products_in_price_range(extraction_id)
        elif choice == 5:
            show_bulk_price_distribution(extraction_id)
        elif choice == 6:
            show_bulk_rating_distribution(extraction_id)
        elif choice == 7:
            show_bulk_brand_frequency(extraction_id)
        elif choice == 8:
            show_bulk_avg_price_by_rating(extraction_id)
        elif choice == 9:
            show_bulk_products_with_review_threshold(extraction_id)
        elif choice == 10:
            show_bulk_products_by_brand(extraction_id)
        elif choice == 11:
            export_bulk_result_to_csv(extraction_id, search_term)
        elif choice == 12:
            print("Returning to main menu.")
            break



In [78]:
def extract_bulk_amazon_data(search_term, driver, page_number, extraction_id, extraction_date):
    """Extract products from current Amazon search page for bulk extraction"""
    products_list = []
    
    # Scroll to load all products on page
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(1)
    driver.execute_script("window.scrollBy(0, 1000);")
    time.sleep(1)
    
    try:
        # Get all product containers on current page
        product_containers = driver.find_elements(By.XPATH, "//div[@data-component-type='s-search-result']")
        print(f"   Found {len(product_containers)} products on page {page_number}")
        
        for i, container in enumerate(product_containers):
            try:
                # Scroll element into view
                driver.execute_script("arguments[0].scrollIntoView(true);", container)
                time.sleep(0.2)
                
                # Get product name
                title_container = container.find_element(By.XPATH, ".//div[@data-cy='title-recipe']")
                try:
                    name_elem = title_container.find_element(By.XPATH, ".//a//h2/span")
                    name = name_elem.text.strip()
                except:
                    try:
                        name_elem = title_container.find_element(By.XPATH, ".//h2//span")
                        name = name_elem.text.strip()
                    except:
                        name = f"Product {i+1} from Page {page_number}"

                # Get product URL
                try:
                    product_link = container.find_element(
                        By.XPATH, 
                        ".//div[@data-cy='title-recipe']//a[contains(@class, 's-line-clamp')][@href]"
                    )
                    url = product_link.get_attribute('href')
                    if 'javascript:void(0)' not in url and ('/dp/' in url or '/sspa/click' in url):
                        if not url.startswith('http'):
                            url = 'https://www.amazon.in' + url
                    else:
                        raise Exception("Found sponsored label link instead of product link")
                        
                except:
                    print(f"❌ Could not find valid product URL for item {i+1}, skipping...")
                    continue

                # Get brand name
                try:
                    brand_elem = container.find_element(By.CSS_SELECTOR, "span.a-size-base-plus.a-color-base")
                    brand_name = brand_elem.text.strip()
                    # Validate brand name
                    if not name:
                        if not brand_name or len(brand_name) < 1 or len(brand_name) > 50:
                            brand_name = "Unknown Brand"
                except:
                    # Fallback to first word of product name if brand selector fails
                    try:
                        first_word = name.split()[0] if name else "Unknown"
                        if len(first_word) > 1:
                            brand_name = first_word
                        else:
                            brand_name = "Unknown Brand"
                    except:
                        brand_name = "Unknown Brand"
                
                # Get price
                try:
                    price_elem = container.find_element(By.XPATH, ".//span[@class='a-price-whole']")
                    price = price_elem.text.strip()
                except:
                    price = "Price not available"

                # Get rating 
                try:
                    rating_elem = container.find_element(By.XPATH, ".//span[contains(@class, 'a-icon-alt')]")
                    rating_text = rating_elem.text.strip() or rating_elem.get_attribute('aria-label') or rating_elem.get_attribute('textContent').strip()
                    if rating_text and ("out of" in rating_text or "stars" in rating_text.lower()):
                        rating = rating_text
                    else:
                        rating = "Rating not available"
                except:
                    rating = "Rating not available"
    
                # Get reviews count
                try:
                    review_elem = container.find_element(By.XPATH, ".//a[contains(@href, '#customerReviews')]//span")
                    review_text = review_elem.text.strip() or review_elem.get_attribute('aria-label')
                    if review_text and ('rating' in review_text.lower() or 'review' in review_text.lower() or any(char.isdigit() for char in review_text)):
                        reviews = review_text
                except:
                    reviews = "Reviews not available"
                # Store product with bulk extraction metadata
                product_info = {
                    'search_term': search_term,
                    'name': name.strip(),
                    'brand_name': brand_name,
                    'price': price,
                    'url': url,
                    'rating': rating,
                    'reviews': reviews,
                    'extraction_id': extraction_id,
                    'page_number': page_number,
                    'extraction_date': extraction_date
                }
  
                products_list.append(product_info)
                
                # Show progress every 10 products
                if len(products_list) % 10 == 0:
                    print(f"     ✓ Extracted {len(products_list)} products from page {page_number}")
                    
            except Exception as e:
                continue  # Skip problematic products
        
        print(f"   ✅ Page {page_number} complete: {len(products_list)} products extracted")
        return products_list
        
    except Exception as e:
        print(f"   ❌ Error extracting from page {page_number}: {e}")
        return []


In [23]:
def perform_bulk_extraction(search_term, target_count=100):
    """Extract products in bulk from Amazon search results across multiple pages"""
    # Generate unique extraction ID
    extraction_id = str(uuid.uuid4())[:8]
    extraction_date = datetime.now().isoformat()
    
    print(f"\n🚀 Starting bulk extraction...")
    print(f"📝 Extraction ID: {extraction_id}")
    print(f"🎯 Target Products: {target_count}")
    
    # Setup ChromeDriver
    options = Options()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-blink-features=AutomationControlled')
    driver = webdriver.Chrome(options=options)
    
    all_products = []
    page_number = 1
    
    try:
        # Navigate to Amazon search
        driver.get('https://www.amazon.in')
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "twotabsearchtextbox"))
        )
        search_box.clear()
        search_box.send_keys(search_term)
        driver.find_element(By.ID, "nav-search-submit-button").click()
        
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@data-component-type='s-search-result']"))
        )
        
        while len(all_products) < target_count:
            print(f"\n📄 Processing Page {page_number}...")
            print(f"📊 Products collected so far: {len(all_products)}/{target_count}")
            
            # Extract products from current page
            page_products = extract_bulk_amazon_data(search_term, driver, page_number, extraction_id, extraction_date)
            
            if not page_products:
                print("⚠️ No more products found on this page.")
                break
            
            all_products.extend(page_products)
            
            # Check if we have enough products
            if len(all_products) >= target_count:
                all_products = all_products[:target_count]  # Trim to exact count
                break
            
            # Try to navigate to next page
            try:
                next_button = driver.find_element(By.XPATH, "//a[contains(@aria-label, 'Go to next page')]")
                if next_button:
                    driver.execute_script("arguments[0].click();", next_button)
                    time.sleep(3)
                    page_number += 1
                    
                    # Wait for new page to load
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.XPATH, "//div[@data-component-type='s-search-result']"))
                    )
                else:
                    print("🔚 No next page available.")
                    break
            except:
                print("🔚 Could not find next page. Extraction complete.")
                break
        
        # Save all products to database
        if all_products:
            print(f"\n💾 Saving {len(all_products)} products to database...")
            save_bulk_products_to_database(all_products)
            
            print(f"\n✅ BULK EXTRACTION COMPLETE!")
            print(f"📊 Total Products Extracted: {len(all_products)}")
            print(f"📄 Pages Processed: {page_number}")
            print(f"🆔 Extraction ID: {extraction_id}")
            
            return extraction_id
        else:
            print("❌ No products were extracted.")
            return None
            
    except Exception as e:
        print(f"❌ Error during bulk extraction: {e}")
        return None
    finally:
        driver.quit()


In [80]:
def bulk_product_analysis():
    """Main function for bulk product extraction and analysis"""
    print("\n📦 BULK PRODUCT ANALYSIS")
    print("="*60)
    
    # Step 1: Get search term from user
    search_term = input("Enter the product name for bulk extraction on Amazon: ").strip()
    if not search_term:
        print("❌ Search term cannot be empty!")
        return
    
    print(f"🔎 Starting bulk extraction for: '{search_term}'")
    
    # Step 2: Get target number of products (with default 100)
    try:
        target_count = int(input("Enter number of products to extract (default 100): ") or "100")
        if target_count <= 0:
            target_count = 100
    except ValueError:
        target_count = 100
    
    print(f"🎯 Target: {target_count} products")
    
    # Step 3: Perform bulk extraction
    extraction_id = perform_bulk_extraction(search_term, target_count)
    
    if extraction_id:
        # Step 4: Show analysis menu
        bulk_analysis_menu(extraction_id, search_term)
    else:
        print("❌ Bulk extraction failed. No data to analyze.")


In [25]:
def get_user_choice(min_choice, max_choice):
    """Get and validate user input choice"""
    while True:
        try:
            choice = int(input(f"Enter your choice ({min_choice}-{max_choice}): "))
            if min_choice <= choice <= max_choice:
                return choice
            else:
                print(f"❌ Invalid choice! Please enter a number between {min_choice} and {max_choice}")
        except ValueError:
            print("❌ Invalid input! Please enter a valid number")
            
def main():
    """Main function to handle the menu system"""
    print("Welcome to Amazon Product Scraper & Price Comparator!")
    # Create database on startup
    print("🗄️ Initializing database...")
    create_database()
    create_comparison_database()
    create_bulk_products_table()

    
    while True:
        # Main Menu
        print("\n" + "="*60)
        print("🛒 AMAZON PRODUCT SCRAPER & PRICE COMPARATOR")
        print("="*60)
        print("1. Today's Deals (Amazon only)")
        print("2. Manual Product Search")
        print("3. View Database")
        print("4. Bulk Product Analysis")
        print("5. Exit") 
        print("="*60)
        
        choice = get_user_choice(1, 5)
        
        if choice == 1:
            todays_deals_scraper()
            
        elif choice == 2:
            # Search Sub-menu
            print("\n" + "-"*50)
            print("📱 MANUAL PRODUCT SEARCH OPTIONS")
            print("-"*50)
            print("1. Amazon Only")
            print("2. Multi-Site (Amazon + Flipkart)")
            print("3. Back to Main Menu")
            print("-"*50)
            
            search_choice = get_user_choice(1, 3)
            
            if search_choice == 1:
                print("🛒 Amazon Only Search - Coming Soon!")
                search_amazon_only()  # To be implemented
                
            elif search_choice == 2:
                print("⚖️ Multi-Site Comparison - Coming Soon!")
                # compare_multisite()  # To be implemented
                compare_amazon_flipkart()
                
            # choice 3 automatically continues to main menu
                
        elif choice == 3:
            print("📊Database Viewer - Coming Soon!")
            view_database_options()

        elif choice == 4:
            print("🆕 Bulk Product Analysis - Coming Soon!")
            bulk_product_analysis()
            
        elif choice == 5:
            print("👋 Thank you for using Amazon Product Scraper!")
            break
        
        input("\nPress Enter to continue...")


In [79]:
 if __name__ == "__main__":
    main()

Welcome to Amazon Product Scraper & Price Comparator!
🗄️ Initializing database...

🛒 AMAZON PRODUCT SCRAPER & PRICE COMPARATOR
1. Today's Deals (Amazon only)
2. Manual Product Search
3. View Database
4. Bulk Product Analysis
5. Exit


Enter your choice (1-5):  4


🆕 Bulk Product Analysis - Coming Soon!

📦 BULK PRODUCT ANALYSIS


Enter the product name for bulk extraction on Amazon:  Laptop Bag


🔎 Starting bulk extraction for: 'Laptop Bag'


...

Returning to main menu.



Press Enter to continue... 



🛒 AMAZON PRODUCT SCRAPER & PRICE COMPARATOR
1. Today's Deals (Amazon only)
2. Manual Product Search
3. View Database
4. Bulk Product Analysis
5. Exit


Enter your choice (1-5):  5


👋 Thank you for using Amazon Product Scraper!
